## Library

In [1]:
import simpy
import numpy.random as random
import pandas as pd
import scipy.stats as st
from collections import namedtuple
from copy import deepcopy

## Floor

In [10]:
def id_generator():
    """Generate unique, global customer ID."""
    i = 1
    while True:
        yield i
        i += 1
        
class IATDistribution:
    def __init__(self, path):
        self.df = pd.read_csv(path)
        self.df["Floor"] = self.df["Floor"].apply(lambda x:x.lstrip('0'))
        
    def getter(self, location, section, direction, floor):
        
        # filter row by mulitple conditions         
        temp = self.df[(self.df['Location'] == location) & (self.df['Section'] == section) & \
                       (self.df['Direction'] == direction) & (self.df['Floor'] == floor)][['Distribution','Parameters']]
        
        # return distribution name and parsed parameters
        return {
            'dist': getattr(st, temp['Distribution'].values[0]),
            'params': self.params_parser(temp['Parameters'].values[0])
        }
    
    @staticmethod
    def params_parser(params_string):
        params = []
        for param in params_string.replace("(", "").replace(")", "").split(", "): 
            params.append(float(param))
        return params

class Queue:
    def __init__(self, env, floor, direction):
        self.env = env
        self.floor = floor
        self.direction = direction
        self.queue_array = []
        self.arrival_event = self.env.event()
        
        # start process
        self.inflow_proc = self.env.process(self.inflow())
        self.outflow_proc = self.env.process(self.outflow())
        
    def inflow(self):
        while True:
            customers = yield self.arrival_event
            print('[INFLOW] Outer Call {} Floor {} '.format(self.floor, 'up' if self.direction == 1 else 'down'))
            
            # disable call if already assigned
            if(len(self.queue_array) == 0):
                global CALL_EVENT
                mission = Mission(direction=self.direction, destination=self.floor)
                CALL_EVENT.succeed(value = mission)
                
                # reactivate event
                CALL_EVENT = self.env.event()
                
            # add customers to waiting queue
            self.queue_array = self.queue_array + customers  
            print('[INFLOW] {}'.format(self.queue_array))
                          
    def outflow(self):
        while True :
            
            # elevator arrives
            availible, elevIndex = yield AAA_EVENT[self.direction][self.floor]

            riders = []
            while (availible > 0) and (len(self.queue_array) > 0):
                riders.append(self.queue_array.pop())
                availible -= 1
                
            # time the customers take to on board
            yield self.env.timeout(len(riders) * random.randint(CONFIG['WALKING_MIN'], CONFIG['WALKING_MAX']))
            
            print('[OUTFLOW] {} People Enters'.format(len(riders)))
                
            # customers on board
            BBB_EVENT[elevIndex].succeed(value=riders)
            BBB_EVENT[elevIndex] = self.env.event()
    
class Floor:
    def __init__(self, env, floor, direction, IAT, DD):
        self.env = env
        self.floor = floor
        self.direction = direction
        
        # statistical data
        self.IAT = IAT
        self.DD = DD if len(DD) == 1 else DD/DD.sum()
        
        # start process
        self.queue = Queue(env, self.floor, self.direction)
        self.source_proc = env.process(self.Source(env))
        
    def Source(self, env):
        while True:
            
            # 1. set inter-arrival time based on given distribution
            t = -1
            while t < 0:
                t = self.IAT['dist'].rvs(*self.IAT['params'][:-2],loc = self.IAT['params'][-2],scale = self.IAT['params'][-1], size = 1)
            yield self.env.timeout(t)     

            # 2. set number of people of arrival group
            customers = []
            for i in range(random.randint(CONFIG['ARRIVAL_MIN'], CONFIG['ARRIVAL_MAX'])):
                
                # 3. set customer destination based on given posibility 
                cid = next(id_gen)
                destination = random.choice(self.DD.index, p=self.DD)
                customers.append(Customer(cid, destination))

            self.queue.arrival_event.succeed(value = customers)
            self.queue.arrival_event = self.env.event()
    
# pd.DataFrame(columns=['cid', 'start_time', 'aboard_time', 'leave_time'])

## Elevator

In [14]:
def displacement(floor1, floor2):
    floor1 = int(floor1) if not 'B' in floor1 else -int(floor1[1:]) + 1 
    floor2 = int(floor2) if not 'B' in floor2 else -int(floor2[1:]) + 1
    return floor2-floor1

class AssignmentError(Exception):
    """Exception : Fail to assign a elevator."""
    pass

class IndexError(Exception):
    """Exception : Wrong Index."""
    pass

Customer = namedtuple("Customer", ["cid", "destination"])
Mission = namedtuple("Mission", ["direction", "destination"])
    
class StopList:
    IDLE = 0; ACTIVE = 1; NA = -1
    
    def __init__(self, FloorList):
        self.floorList = deepcopy(FloorList)
        self._list = {
             1: [0] * (displacement(FloorList[0], FloorList[len(FloorList)-1])+1),
            -1: [0] * (displacement(FloorList[0], FloorList[len(FloorList)-1])+1)
        }
        
        # dictionary for indexing
        self.index = {
             1:{ floor:index for index, floor in enumerate(         FloorList ) },
            -1:{ floor:index for index, floor in enumerate(reversed(FloorList)) }
        }
        self.reversed_index = {
             1:{ index:floor for index, floor in enumerate(FloorList) },
            -1:{ index:floor for index, floor in enumerate(reversed(FloorList)) }
        }
    
    def __str__(self):
        string = 'Stop List: \n  up   down \n'
        sss = ['{} [{}] [{}] {}\n'.format(f, i, j, f) for f, i, j in zip(self.floorList, self._list[1], self._list[-1])]
        print('!!!!!!!!!!!!!!!!111',sss)
        string.join(sss)
        return string
    
    def isEmpyty(self):
        for i, j in zip(self._list[1], self._list[-1]):
            if i == StopList.ACTIVE or j == StopList.ACTIVE:
                return False
        return True
    
    def pushOuter(self, elev, direction, floor):
        currentIndex = self.index[direction][floor]
        
        # illegal index
        if self._list[direction][currentIndex] == StopList.NA:
            raise IndexError()
        
        # add outer call to stoplist
        self._list[direction][currentIndex] = StopList.ACTIVE
        
    def pushInner(self, elev, destination):
        currentIndex = self.index[elev.direction][destination]
        
        # illegal index
        if self._list[elev.direction][currentIndex] == StopList.NA:
            raise IndexError()
            
        # add inner call to stoplist
        self._list[elev.direction][currentIndex] = StopList.ACTIVE
    
    def pop(self, elev):
        currentIndex = self.index[elev.direction][elev.current_floor]
        
        # illegal index
        if self._list[elev.direction][currentIndex] == StopList.NA:
            raise IndexError()
        
        # remove target floor from stoplist
        self._list[elev.direction][currentIndex] = StopList.IDLE
    
    def next_target(self, elev):
        v = 3
        if v == 3:
            # same direction, front
            curr_index = self.index[elev.direction][elev.current_floor]
            for i, state in enumerate(self._list[elev.direction][curr_index:]):
                if state == StopList.ACTIVE:
                    return self.reversed_index[elev.direction][i+curr_index]

            # different direction
            for i, state in enumerate(self._list[-elev.direction]):
                if state == StopList.ACTIVE:
                    return self.reversed_index[-elev.direction][i]

            # same direction, back
            for i, state in enumerate(self._list[elev.direction][:curr_index]):
                if state == StopList.ACTIVE:
                    return self.reversed_index[elev.direction][i]
            
        if v == 2:
            peak = None   
        
            # same direction, front
            curr_index = self.index[elev.direction][elev.current_floor]
            for i, state in enumerate(self._list[elev.direction][curr_index:]):
                if state == StopList.ACTIVE:
                    return self.reversed_index[elev.direction][i+curr_index]

                # compute ligetimate peak
                if state != StopList.NA:
                    peak = self.reversed_index[elev.direction][i+curr_index]

            # different direction
            for i, state in enumerate(self._list[-elev.direction]):
                if state == StopList.ACTIVE:
                    return peak

            # same direction, back
            for i, state in enumerate(self._list[elev.direction][:curr_index]):
                if state == StopList.ACTIVE:
                    return self.reversed_index[elev.direction][i]
        
        return None

class Elevator:
    def __init__(self, env, elevIndex, failAssignment, floorList):
        self.env = env
        self.elevIndex = elevIndex
        self.capacity = CONFIG['ELEV_CAPACITY']
        self.riders = []
        self.failAssignment = failAssignment
        
        # schedule list
        self.stop_list = StopList(floorList)
        
        # initial states
        self.current_floor = '1'
        self.direction = 0
        self.assign_event = self.env.event()
        
        # start process
        self.env.process(self.idle())
            
    def idle(self):
        print('[IDLE] Elevator {} Activated'.format(self.elevIndex))
        
        while True:
            
            # first assignment
            mission = yield self.assign_event
            
            # reactivate
            self.assign_event = self.env.event()
            
            print('[IDLE] Outer Call: {}'.format(mission))
            
            # push outer call
            self.stop_list.pushOuter(self, mission.direction, mission.destination)

            # determine initial direction
            if displacement(self.current_floor, mission.destination) > 0:
                self.direction = 1
            elif displacement(self.current_floor, mission.destination) < 0:
                self.direction = -1
            else:
                # set the direction of elevator to that of mission
                self.direction = mission.direction    
            
            # start onMission process
            yield self.env.process(self.onMission())
            
            print('[IDLE] Elevator {} Stopped.'.format(self.elevIndex))
            
            # return to IDLE state
            self.direction = 0
#             self.resubmit()
    
    def onMission(self):
        while not self.stop_list.isEmpyty():
            
            print('[ONMISSION] On MISSION')
            print(self.stop_list)
            
            nextTarget = self.stop_list.next_target(self)
            print('[ONMISSION] NEXT TARGET {}'.format(nextTarget))
        
            moving_proc = self.env.process(self.moving( nextTarget, self.current_floor ))
            
            while self.current_floor != nextTarget:
                value = yield self.assign_event | moving_proc
                
                if self.assign_event.triggered:
                    mission = value[self.assign_event]
                    self.assign_event = self.env.event()
                    print('[ONMISSION] New Assigenment {}'.format(mission))
                    
                    before = nextTarget
                    self.stop_list.pushOuter(self, mission.direction, mission.destination)
                    
                    nextTarget = self.stop_list.next_target(self)
                    print('[ONMISSION] NEXT TARGET {}'.format(nextTarget))
                    if before != nextTarget:
                        
                        moving_proc.interrupt()
                        moving_proc = self.env.process(self.moving( nextTarget, self.current_floor ))
            
            print('[ONMISSION] Arrive At {} Floor'.format(self.current_floor))
            
            yield self.env.process(self.serving())
            print('[Afer Serving]',self.riders)
        
    def moving(self, destination, source):
        """source is needed to account for the acceleration and deceleration rate of the elevator"""
        print('[MOVING] Moving Process Started')
        try:   
            while self.current_floor != destination:
                
                # determine traveling time for 1 floor
                t = self.travelingTime(destination, self.current_floor, source)
                yield self.env.timeout(t)
                
                # advance 1 floor
                self.current_floor = self.forwards(self.current_floor, self.direction)
                
                print('[MOVING] Update To {} Floor'.format(self.current_floor))
        except simpy.Interrupt as i:
            print('[MOVING] Interrupted')
            print(self.stop_list)
            
        
    def serving(self):
        
        # remove from schedule
        self.stop_list.pop(self)
        print('pop:',self.stop_list)
        
        # customers leave
        leaveCount = 0
        for i in range(len(self.riders)-1, -1, -1):
            if self.riders[i].destination == self.current_floor:
                self.riders.pop(i)
                leaveCount += 1
        print('[SERVING] {} Customers Leave'.format(countCus))
        
        # exclude 'peak' condition
        if not((self.current_floor == FloorList[-1] and self.direction == 1) or \
               (self.current_floor == FloorList[0] and self.direction == -1)):
            
            # elevator arrive
            AAA_EVENT[self.direction][self.current_floor].succeed( value = (self.capacity-len(self.riders), self.elevIndex))
            AAA_EVENT[self.direction][self.current_floor] = self.env.event()  

            # customers on board
            riders = yield BBB_EVENT[self.elevIndex]

            print('[SERVING] Customers Aboard: \n  ', riders)

            # add inner calls
            for customer in riders:
                self.stop_list.pushInner(self, customer.destination)
            self.riders = self.riders + riders
            
            print(self.stop_list)
        
        # determine direction
        nextTarget = self.stop_list.next_target(self)
        print('[SERVING] NEXT TARGET {}'.format(nextTarget))
        if nextTarget is None:
            return
        else:
            if displacement(self.current_floor, nextTarget) > 0:
                self.direction = 1
            elif displacement(self.current_floor, nextTarget) < 0:
                self.direction = -1
            else:
                
                # make a turn
                self.direction = -1*self.direction
                self.stop_list.pop(self)
                print('p2p:',self.stop_list)             
                
                # customers on board
                self.boarding()
    
        
    def resubmit(self):
        for i in range(len(self.failAssignment)):
            mission = self.failAssignment.pop()
            global RESUBMIT_EVENT
            RESUBMIT_EVENT.succeed(value=mission)
            RESUBMIT_EVENT = self.env.event()
    
    def travelingTime(self, destination, current, source):
        # acceleration should be considered
        return 10
    
    def boarding(self):
        # boarding
        AAA_EVENT[self.direction][self.current_floor].succeed( value = (self.capacity-len(self.riders), self.elevIndex))
        AAA_EVENT[self.direction][self.current_floor] = self.env.event()  

        riders = yield BBB_EVENT[self.elevIndex]

        print('[SERVING] Customers Aboard: \n  ', riders)

        # new customers
        for customer in riders:
            self.stop_list.pushInner(self, customer.destination)
        print(self.stop_list)
        self.riders = self.riders + riders
        
    @staticmethod
    def mission_priority(mission):
        floor = mission.destination
        direction = mission.direction
        index = int(floor) if not 'B' in floor else -int(floor[1:]) + 1
        index = index*direction
        return index  
    
    @staticmethod
    def forwards(floor, direction):
        floor = int(floor) if not 'B' in floor else -int(floor[1:]) + 1
        floor += direction
        floor = str(floor) if floor > 0 else "B{}".format(abs(floor-1))
        return floor
        
class ElevatorController:
    def __init__(self, env, elevatorList, floorList):
        self.env = env
        self.elevatorList = elevatorList
        self.failAssignment = []
        self.elevators = dict()
        for elevatorName in elevatorList:
            self.elevators[elevatorName] = Elevator(env, elevatorName, self.failAssignment, floorList)
        self.env.process(self.assignCalls())
    
    def assignCalls(self):
        while True:
            mission = yield CALL_EVENT# | RESUBMIT_EVENT
            
            try:
#                 candidate = self.bestCandidate(mission.direction, mission.destination)
                candidate = self.elevators['3001'].elevIndex
        
                self.elevators[candidate].assign_event.succeed(value = mission)
                
            except AssignmentError:
                print('Fail to Assign')
                
    def bestCandidate(self, direction, source):
        """Assignment Policy"""
        minDistance = 50
        bestElevator = str()
        for elevator in self.elevators.values():
            if (minDistance > abs(displacement(source, elevator.current_floor)) and \
               ((direction == elevator.direction and elevator.current_floor*direction < direction*source) \
                or elevator.direction == 0)):
                    minDistance = abs(displacement(source, elevator.current_floor))
                    bestElevator = elevator.elevIndex
            
        if minDistance == 50:
            self.failAssignment.append(Mission(direction=direction, destination=source))
            raise AssignmentError()
            
        return bestElevator


In [15]:
# Inter-Arrival Time Distribution
IATD = IATDistribution('./BestFitDistribution - 複製.csv')

# Destination Distribution
DD = pd.read_csv('./FloorRatio_NHB.csv').iloc[:,1:].set_index('from').iloc[4:8,4:8]

# Location & Time Section
Location = "北棟病床"; TimeSection = 2

# Floor List
FloorList = ['1','2', '3', '4']

# Elevator ID
ElevatorList = ['3001']

# Enviornment Variable
env = simpy.Environment()
id_gen = id_generator()
CONFIG = {
    'ARRIVAL_MAX': 3,
    'ARRIVAL_MIN': 1,    
    'WALKING_MAX': 3,
    'WALKING_MIN': 1,
    'RANDOM_SEED': 50,
    'ELEV_CAPACITY': 15
    
}
random.seed(CONFIG['RANDOM_SEED'])

# Global Event
CALL_EVENT = env.event()
RESUBMIT_EVENT = env.event()
AAA_EVENT = { 1: { i : env.event() for i in FloorList}, -1: { i : env.event() for i in FloorList} }
BBB_EVENT = { i: env.event() for i in ElevatorList }

# process
floors_upward   = [ Floor(env, i,  1, IATD.getter(Location, TimeSection,   'up', i), DD.loc[i][i.lstrip("0"):]) for i in FloorList[ :-1]]
floors_downward = [ Floor(env, i, -1, IATD.getter(Location, TimeSection, 'down', i), DD.loc[i][:i.lstrip("0")]) for i in FloorList[1:  ]]
elevator_ctrl = ElevatorController(env, ElevatorList, FloorList)

env.run(until=10000)

[IDLE] Elevator 3001 Activated
[INFLOW] Outer Call 1 Floor up 
[INFLOW] [Customer(cid=1, destination='3'), Customer(cid=2, destination='3')]
[IDLE] Outer Call: Mission(direction=1, destination='1')
[ONMISSION] On MISSION
!!!!!!!!!!!!!!!!111 ['1 [1] [0] 1\n', '2 [0] [0] 2\n', '3 [0] [0] 3\n', '4 [0] [0] 4\n']
Stop List: 
  up   down 

[ONMISSION] NEXT TARGET 1
[ONMISSION] Arrive At 1 Floor
[MOVING] Moving Process Started
pop: !!!!!!!!!!!!!!!!111 ['1 [0] [0] 1\n', '2 [0] [0] 2\n', '3 [0] [0] 3\n', '4 [0] [0] 4\n']
Stop List: 
  up   down 

[SERVING] 0 Customers Leave
NOt PeAk
[OUTFLOW] 2 People Enters
[SERVING] Customers Aboard: 
   [Customer(cid=2, destination='3'), Customer(cid=1, destination='3')]
!!!!!!!!!!!!!!!!111 ['1 [0] [0] 1\n', '2 [0] [0] 2\n', '3 [1] [0] 3\n', '4 [0] [0] 4\n']
Stop List: 
  up   down 

[SERVING] NEXT TARGET 3
GOING UP
[Afer Serving] [Customer(cid=2, destination='3'), Customer(cid=1, destination='3')]
[ONMISSION] On MISSION
!!!!!!!!!!!!!!!!111 ['1 [0] [0] 1\n',

[INFLOW] Outer Call 4 Floor down 
[INFLOW] [Customer(cid=6, destination='1'), Customer(cid=7, destination='3'), Customer(cid=14, destination='3'), Customer(cid=15, destination='1'), Customer(cid=33, destination='3'), Customer(cid=43, destination='3'), Customer(cid=48, destination='1'), Customer(cid=49, destination='1'), Customer(cid=50, destination='1'), Customer(cid=86, destination='1'), Customer(cid=87, destination='3'), Customer(cid=89, destination='1'), Customer(cid=90, destination='1'), Customer(cid=100, destination='1'), Customer(cid=123, destination='1'), Customer(cid=124, destination='1')]
[INFLOW] Outer Call 2 Floor down 
[INFLOW] [Customer(cid=111, destination='1'), Customer(cid=112, destination='1'), Customer(cid=118, destination='1'), Customer(cid=121, destination='1'), Customer(cid=122, destination='1'), Customer(cid=125, destination='1')]
[INFLOW] Outer Call 1 Floor up 
[INFLOW] [Customer(cid=3, destination='3'), Customer(cid=10, destination='4'), Customer(cid=13, destina